In [2]:
!pip install opencv-python
!pip install pafy
!pip install torch
!pip install torchvision
!pip install youtube-dl==2020.12.2

import cv2
import pafy # pafy allows us to read videos from youtube.
import torch
import numpy as np
from time import time

In [7]:
class YoutubeYolo5ObjectDetection:
    """
    Class implements Yolo5 model to make inferences on a youtube video using Opencv2.
    """

    def __init__(self, url, out_file="Cattle_Labeled_Video.avi"):
        """
        Initializes the class with youtube url and output file.
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """
        self._URL = url
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """
        play = pafy.new(self._URL).streams[-1]
        assert play is not None
        return cv2.VideoCapture(play.url) 

    def load_model(self):
        """
        Loads Yolo5 model from pytorch hub.
        :return: Trained Pytorch model.
        """
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        allowed_classes = set([0, 16, 17, 18, 19])
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()
        neg = 0
        for i in range(len(labels)):
            if labels[i] not in allowed_classes:
                labels[i] = -1
                neg += 1

        number_of_objects = str(len(labels)-neg)
        return labels, cord, number_of_objects

    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord, freq = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            if labels[i] == -1:
                continue
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, ".", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 5, bgr, 2)
                cv2.putText(frame, freq, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """
        player = self.get_video_from_url()
        assert player.isOpened()
        
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))
        
        skip_frame = 5 #skipping every 5 frames to decrease compute time
        
        while True:
            start_time = time()
            ret, frame = player.read()
            if skip_frame == 1: 
                skip_frame = 5
                continue
            skip_frame -= 1
            assert ret
            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()
            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")

            out.write(frame)

In [8]:
URL = "https://www.youtube.com/watch?v=Mb09RIvm76E&ab_channel=IndianCow"
output_file_name = "Cattle_Labeled_Video.avi"

a = YoutubeYolo5ObjectDetection(URL, output_file_name)
a()
# output video is stored in same folder

Using cache found in C:\Users\Krishika/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-12-2 Python-3.9.13 torch-1.13.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Frames Per Second : 2.2075055187637966
Frames Per Second : 2.8818443804034586
Frames Per Second : 3.3222591362126246
Frames Per Second : 2.8901734104046244
Frames Per Second : 3.0769230769230766
Frames Per Second : 2.6315789473684212
Frames Per Second : 2.3866348448687353
Frames Per Second : 3.021148036253776
Frames Per Second : 3.03951367781155
Frames Per Second : 2.865329512893983
Frames Per Second : 2.8328611898017
Frames Per Second : 2.976190476190476
Frames Per Second : 2.824858757062147
Frames Per Second : 2.9673590504451037
Frames Per Second : 2.3094688221709005
Frames Per Second : 2.5125628140703515
Frames Per Second : 2.7777777777777777
Frames Per Second : 3.115264797507788
Frames Per Second : 2.8901734104046244
Frames Per Second : 2.8409090909090913
Frames Per Second : 2.808988764044944
Frames Per Second : 3.236245954692557
Frames Per Second : 2.949852507374631
Frames Per Second : 2.6455026455026456
Frames Per Second : 2.463054187192118
Frames Per Second : 2.857142857142857
F

Frames Per Second : 3.1746031746031744
Frames Per Second : 2.6041666666666665
Frames Per Second : 2.801120448179272
Frames Per Second : 3.3444816053511706
Frames Per Second : 2.9850746268656714
Frames Per Second : 2.6525198938992043
Frames Per Second : 3.558718861209964
Frames Per Second : 3.0769230769230766
Frames Per Second : 2.8490028490028494
Frames Per Second : 2.5974025974025974
Frames Per Second : 3.058103975535168
Frames Per Second : 2.487562189054726
Frames Per Second : 2.808988764044944
Frames Per Second : 2.7472527472527473
Frames Per Second : 3.144654088050314
Frames Per Second : 3.3333333333333335
Frames Per Second : 2.923976608187134
Frames Per Second : 2.857142857142857
Frames Per Second : 2.710027100271003
Frames Per Second : 3.03951367781155
Frames Per Second : 3.0864197530864197
Frames Per Second : 2.976190476190476
Frames Per Second : 2.557544757033248
Frames Per Second : 2.923976608187134
Frames Per Second : 2.7548209366391188
Frames Per Second : 3.1746031746031744


Frames Per Second : 3.0303030303030303
Frames Per Second : 2.9585798816568047
Frames Per Second : 2.710027100271003
Frames Per Second : 2.8985507246376816
Frames Per Second : 3.144654088050314
Frames Per Second : 2.5839793281653747
Frames Per Second : 3.3333333333333335
Frames Per Second : 3.1746031746031744
Frames Per Second : 2.865329512893983
Frames Per Second : 3.278688524590164
Frames Per Second : 3.4602076124567476
Frames Per Second : 2.7472527472527473
Frames Per Second : 3.546099290780142
Frames Per Second : 2.8901734104046244
Frames Per Second : 2.9673590504451037
Frames Per Second : 3.1746031746031744
Frames Per Second : 3.134796238244514
Frames Per Second : 3.048780487804878
Frames Per Second : 3.367003367003367
Frames Per Second : 3.2679738562091503
Frames Per Second : 3.058103975535168
Frames Per Second : 3.3444816053511706
Frames Per Second : 2.570694087403599
Frames Per Second : 2.2675736961451247
Frames Per Second : 2.7027027027027026
Frames Per Second : 2.9154518950437

Frames Per Second : 2.525252525252525
Frames Per Second : 3.623188405797101
Frames Per Second : 3.236245954692557
Frames Per Second : 3.1847133757961785
Frames Per Second : 2.906976744186047
Frames Per Second : 3.2467532467532467
Frames Per Second : 3.067484662576687
Frames Per Second : 3.194888178913738
Frames Per Second : 3.0303030303030303
Frames Per Second : 2.73972602739726
Frames Per Second : 3.194888178913738
Frames Per Second : 2.6737967914438503
Frames Per Second : 3.194888178913738
Frames Per Second : 3.115264797507788
Frames Per Second : 2.7548209366391188
Frames Per Second : 3.1847133757961785
Frames Per Second : 3.236245954692557
Frames Per Second : 2.8409090909090913
Frames Per Second : 3.134796238244514
Frames Per Second : 2.8818443804034586
Frames Per Second : 3.1645569620253164
Frames Per Second : 3.1645569620253164
Frames Per Second : 2.680965147453083
Frames Per Second : 2.873563218390805
Frames Per Second : 3.144654088050314
Frames Per Second : 3.021148036253776
Fra

...

Frames Per Second : 2.7548209366391188
Frames Per Second : 2.7472527472527473
Frames Per Second : 2.6666666666666665
Frames Per Second : 2.6595744680851063
Frames Per Second : 2.7548209366391188
Frames Per Second : 2.6954177897574123
Frames Per Second : 2.7472527472527473
Frames Per Second : 2.6525198938992043
Frames Per Second : 2.6525198938992043
Frames Per Second : 2.7027027027027026
Frames Per Second : 2.6737967914438503
Frames Per Second : 2.785515320334262
Frames Per Second : 2.710027100271003
Frames Per Second : 2.808988764044944
Frames Per Second : 2.6595744680851063
Frames Per Second : 2.8490028490028494
Frames Per Second : 2.564102564102564
Frames Per Second : 2.73224043715847
Frames Per Second : 2.770083102493075
Frames Per Second : 2.6315789473684212
Frames Per Second : 2.6041666666666665
Frames Per Second : 2.7777777777777777
Frames Per Second : 2.6954177897574123
Frames Per Second : 2.8818443804034586
Frames Per Second : 2.6455026455026456
Frames Per Second : 2.7932960893

Frames Per Second : 2.6178010471204187
Frames Per Second : 2.6666666666666665
Frames Per Second : 2.808988764044944
Frames Per Second : 2.8328611898017
Frames Per Second : 2.770083102493075
Frames Per Second : 2.801120448179272
Frames Per Second : 2.6954177897574123
Frames Per Second : 2.857142857142857
Frames Per Second : 2.6954177897574123
Frames Per Second : 2.638522427440633
Frames Per Second : 2.6595744680851063
Frames Per Second : 2.785515320334262
Frames Per Second : 2.7027027027027026
Frames Per Second : 2.638522427440633
Frames Per Second : 2.785515320334262
Frames Per Second : 2.7027027027027026
Frames Per Second : 2.7777777777777777
Frames Per Second : 2.6595744680851063
Frames Per Second : 2.8328611898017
Frames Per Second : 2.6315789473684212
Frames Per Second : 2.1929824561403506
Frames Per Second : 2.7548209366391188
Frames Per Second : 2.710027100271003
Frames Per Second : 2.7777777777777777
Frames Per Second : 2.73972602739726
Frames Per Second : 2.941176470588235
Fram

Frames Per Second : 3.0769230769230766
Frames Per Second : 3.3783783783783785
Frames Per Second : 3.3783783783783785
Frames Per Second : 2.941176470588235
Frames Per Second : 3.4843205574912894
Frames Per Second : 3.278688524590164
Frames Per Second : 2.6881720430107525
Frames Per Second : 3.4965034965034967
Frames Per Second : 3.2679738562091503
Frames Per Second : 3.03951367781155
Frames Per Second : 2.9850746268656714
Frames Per Second : 3.144654088050314
Frames Per Second : 3.0769230769230766
Frames Per Second : 3.0303030303030303
Frames Per Second : 3.2258064516129035
Frames Per Second : 3.2894736842105265
Frames Per Second : 3.2051282051282053
Frames Per Second : 3.2679738562091503
Frames Per Second : 3.048780487804878
Frames Per Second : 3.1746031746031744
Frames Per Second : 3.4722222222222223
Frames Per Second : 3.1545741324921135
Frames Per Second : 3.03951367781155
Frames Per Second : 3.3444816053511706
Frames Per Second : 3.0303030303030303
Frames Per Second : 2.80112044817

Frames Per Second : 2.976190476190476
Frames Per Second : 2.6666666666666665
Frames Per Second : 2.793296089385475
Frames Per Second : 3.367003367003367
Frames Per Second : 3.2258064516129035
Frames Per Second : 2.7247956403269757
Frames Per Second : 3.021148036253776
Frames Per Second : 2.857142857142857
Frames Per Second : 2.785515320334262
Frames Per Second : 2.8490028490028494
Frames Per Second : 3.012048192771084
Frames Per Second : 2.857142857142857
Frames Per Second : 3.236245954692557
Frames Per Second : 2.9585798816568047
Frames Per Second : 2.6666666666666665
Frames Per Second : 3.1847133757961785
Frames Per Second : 3.236245954692557
Frames Per Second : 3.134796238244514
Frames Per Second : 3.2894736842105265
Frames Per Second : 2.6881720430107525
Frames Per Second : 2.9585798816568047
Frames Per Second : 2.4096385542168677
Frames Per Second : 2.5906735751295336
Frames Per Second : 2.865329512893983
Frames Per Second : 2.717391304347826
Frames Per Second : 2.5773195876288657

AssertionError: 